In [8]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Functipn to split data into training, validation and test sets
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import glob   # The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. No tilde expansion is done, but *, ?, and character ranges expressed with [] will be correctly matched.
import os   # miscellneous operating system interfaces. This module provides a portable way of using operating system dependent functionality. If you just want to read or write a file see open(), if you want to manipulate paths, see the os.path module, and if you want to read all the lines in all the files on the command line see the fileinput module.
import random       
from tqdm import tqdm 
from tqdm.notebook import tqdm_notebook
import datetime
import time
from tabulate import tabulate

# Torch
import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import torchinfo


from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve,log_loss, accuracy_score, f1_score
from sklearn.metrics import average_precision_score,roc_auc_score
import os
import time
from time import time
import datetime
import pandas as pd
import numpy as np
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.adapt import MLkNN

# CMAP (extracting relevant transcriptomic profiles)
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.subset_gctoo as sg
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
import time
import joblib

from sklearn.decomposition import PCA,FactorAnalysis
from sklearn.preprocessing import StandardScaler,QuantileTransformer
from sklearn.metrics import precision_recall_curve,log_loss
from sklearn.metrics import average_precision_score,roc_auc_score
from sklearn.feature_selection import VarianceThreshold
import os
import pandas as pd
import numpy as np


In [9]:
now = str(datetime.now())
print(now)

2023-01-20 08:57:06.216875


In [10]:
# ----------------------------------------- hyperparameters ---------------------------------------#
# Hyperparameters
#testing = False # decides if we take a subset of the data
max_epochs = 75 # number of epochs we are going to run 
# apply_class_weights = True # weight the classes based on number of compounds
using_cuda = True # to use available GPUs
world_size = torch.cuda.device_count()
#---------------------------------------------------------------------------------------------------#
start = time.time()
#now = datetime.datetime.now()
#now = now.strftime("%d_%m_%Y-%H:%M:%S")

1. Downloading all relevant data frames and csv files
2. extract the correct transcriptomic profiles
3. prepare train vs test columns using one hot encoding
4. evaluate results
5. save results

## 1. Downloading all relevant data frames and csv files 

In [11]:
# clue column metadata with columns representing compounds in common with SPECs 1 & 2
clue_sig_in_SPECS = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_sig_in_SPECS1&2.csv', delimiter = ",")

In [12]:
# clue row metadata with rows representing transcription levels of specific genes
clue_gene = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_geneinfo_beta.txt', delimiter = "\t")

In [13]:
# the training set, what transcriptomic profiles/compounds are included in only the training set
L1000_training = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_training_set.csv', delimiter = ",")

In [14]:
# the training set, what transcriptomic profiles/compounds are included in only the training set
L1000_validation = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_valid_set.csv', delimiter = ",")

In [49]:
L1000_training.head()

,Compound ID,sig_id,moa
0,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,0
1,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,0
2,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,0
3,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,0
4,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,0


In [16]:
num_classes = len(L1000_training['moa'].unique())
num_classes

10

In [17]:
def variance_threshold(x_train, x_val):
    """
    This function perform feature selection on the data, i.e. removes all low-variance features below the
    given 'threshold' parameter.
    
    Args:
            x_fold_train: K-fold train data with only phenotypic/morphological features and PCs - pandas 
            dataframe.
            x_fold_val: K-fold validation data with only phenotypic/morphological features and PCs - pandas 
            dataframe.
            df_test_x_copy: test data - pandas dataframe with only phenotypic/morphological features and PCs.
    
    Returns:
            x_fold_train: K-fold train data after feature selection - pandas dataframe.
            x_fold_val: K-fold validation data after feature selection - pandas dataframe.
            df_test_x_copy: test data - pandas dataframe after feature selection - pandas dataframe.
    
    """
    var_thresh = VarianceThreshold(threshold = 2) # sets a variance threshold
    var_thresh.fit(x_train) # learn empirical variances from X
    x_train = x_train.loc[:,var_thresh.variances_ > 2] # locate all variance thresholds above 0.8, keep those columns
    x_val = x_val.loc[:,var_thresh.variances_ > 2]
    return x_train, x_val

In [18]:
# preparing the gctoo dataframe to extract from
# choosing only landmark genes
def tprofiles_gc_too_func(L1000_data, clue_gene):
    '''
    
    Input:
    L1000: column meta data from clue.io that only includes training/test data
    clue_gene: row meta data from clue.io transcriptomic profiles
    
    Output:
    parsed gctoo file with all of the transcriptomic profiles'''

    clue_gene["gene_id"] = clue_gene["gene_id"].astype(str)
    landmark_gene_row_ids = clue_gene["gene_id"][clue_gene["feature_space"] == "landmark"]

    # get all samples (across all cell types, doses, and other treatment conditions) with certain MoA
    profile_ids = L1000_data["sig_id"]
    tprofiles_gctoo = parse("/scratch2-shared/erikep/level5_beta_trt_cp_n720216x12328.gctx", 
                                    cid= profile_ids, 
                                    rid = landmark_gene_row_ids)

    ### 2. copy the metadata from those samples
    # first, we need to subset all the metadata information from our larger metadata DataFrame
    # col
  
    '''
    tprofiles_sig_id_info = L1000_training

    # row 
    tprofiles_gene_id_info = clue_gene[clue_gene["feature_space"] == "landmark"]

    tprofiles_sig_id_info.set_index("sig_id", inplace=True)
    # now the data frame is indexed by sig_ids consistent with those of the data_df:
    tprofiles_sig_id_info.index

    tprofiles_gene_id_info.set_index("gene_id", inplace=True)
    # now the data frame is indexed by sig_ids consistent with those of the data_df:
    tprofiles_gene_id_info.index

    # set the relevant annotations as a col_metadata_df: 
    tprofiles_gctoo.col_metadata_df = tprofiles_sig_id_info

    # set the relevant annotations as a row_metadata_df: 
    tprofiles_gctoo.row_metadata_df = tprofiles_gene_id_info
    '''
    return tprofiles_gctoo

In [19]:
def extract_tprofile(idx):
    '''returns transcriptomic profile of of specific ID with in the form of a torch tensor'''
    # extract unique column name from L1000 data
    # use it to parse the gctoo file
    
    tprofile_id =  profiles_gc_too.col_metadata_df.iloc[idx]
    tprofile_id_sig = [tprofile_id.name] 
    tprofile_gctoo = sg.subset_gctoo(profiles_gc_too, cid= tprofile_id_sig) 
    #return torch.tensor(tprofile_gctoo.data_df.values.astype(np.float32)) 
    return np.asarray(tprofile_gctoo.data_df.values.astype(np.float32))    

In [20]:
def splitting(df):
    '''Splitting data into two parts:
    1. input : the pointer showing where the transcriptomic profile is  
    2. target one hot : labels (the correct MoA) '''
    
    # one-hot encoding labels
     # creating tensor from all_data.df
   #target = torch.tensor(df['moa'].values.astype(np.int64))

    # For each row, take the index of the target label
    # (which coincides with the score in our case) and use it as the column index to set the value 1.0.” 
    #target_onehot = torch.zeros(target.shape[0], num_classes)
    #target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
    target_onehot = df['moa']
    input =  df.drop('moa', axis = 1)
    
    
    return input, target_onehot


In [21]:
def np_array_transform(profiles_gc_too):
    '''
    Takes a .gctoo and extracts the correct profile, transforms the profile into a numpy array and then places it into a pandas data_frame.

    Input:

    Output:
    pandas dataframe
    '''
    rows = []
    for i in range(profiles_gc_too.data_df.shape[1]):
        rows.append(extract_tprofile(i))
    np_array =  np.asarray(rows)
    np_array = np_array.squeeze()
    df = pd.DataFrame(np_array)
    return df

In [22]:
'''rows = []
for i in range(profiles_gc_too.data_df.shape[1]):
    rows.append(extract_tprofile(i))
np_df = pd.DataFrame(rows)'''

'rows = []\nfor i in range(profiles_gc_too.data_df.shape[1]):\n    rows.append(extract_tprofile(i))\nnp_df = pd.DataFrame(rows)'

beloww takes a long time (8 min)

# Training

In [23]:
profiles_gc_too = tprofiles_gc_too_func(L1000_training, clue_gene)

In [24]:
input_df, labels = splitting(L1000_training) 

In [33]:
df_train = np_array_transform(profiles_gc_too)


In [34]:
np.save('/scratch2-shared/erikep/data_splits_npy/10_moas_train', df_train)

In [25]:
labels

0        0
1        0
2        0
3        0
4        0
        ..
14110    8
14111    8
14112    8
14113    8
14114    8
Name: moa, Length: 14115, dtype: int64

In [26]:
labels_train_str = labels.apply(str)


In [27]:
'''mlb = MultiLabelBinarizer()
mlb.fit(labels_str)'''


'mlb = MultiLabelBinarizer()\nmlb.fit(labels_str)'

In [28]:
'''labels_touse = mlb.fit_transform(labels_str)'''

'labels_touse = mlb.fit_transform(labels_str)'

In [29]:
'''labels_str'''

'labels_str'

In [31]:
#df_train_vs.shape

In [32]:
# Test with muli-label MlKNN to see if it works
classifier = RandomForestClassifier()
classifier.fit(df_train_vs, labels_train_str)

NameError: name 'df_train_vs' is not defined

### Using a version of grid search to optimize random forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
'''taken from https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74'''
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
rf_random = RandomizedSearchCV(estimator = classifier, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(df_train, labels_train_str)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [ ]:
best_random = rf_random.best_estimator_


In [ ]:
#joblib.dump(best_random, '/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/Erik_Models/pickles/Random_Forest_pickles/RF' + now)

['/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/Erik_Models/pickles/Random_Forest_pickles/RF2023-01-18 15:52:55.679817']

In [ ]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': False}

In [ ]:
rf_random

In [ ]:
print(classifier.score(df_train, labels_train_str))


1.0


In [ ]:
df_train_numpy = df_train.to_numpy()
labels_train = labels.to_numpy()

In [ ]:
labels_train.shape

(14115,)

In [ ]:
df_train_numpy.shape

(14115, 978)

# Validation

In [35]:
profiles_gc_too = tprofiles_gc_too_func(L1000_validation, clue_gene)

In [36]:
df_val = np_array_transform(profiles_gc_too)

In [37]:
np.save('/scratch2-shared/erikep/data_splits_npy/10_moas_val', df_val)

In [38]:
input_df_val, labels_val = splitting(L1000_validation) 
labels_val

0       7
1       7
2       7
3       7
4       7
       ..
1973    2
1974    2
1975    2
1976    2
1977    2
Name: moa, Length: 1978, dtype: int64

In [ ]:
labels_val_str = labels_val.apply(str)

Using variance threshold

In [ ]:
df_train__vs, df_val_vs = variance_threshold(df_train, df_val)

In [ ]:
df_val_vs.shape

(1978, 713)

In [ ]:
df_train__vs.shape

(14115, 713)

In [ ]:
classifier.fit(df_train__vs, labels_train_str)

RandomForestClassifier()

In [ ]:
predictions = classifier.predict(df_val_vs)

In [ ]:
accuracy_score(labels_val_str, predictions)

0.2204246713852376

In [ ]:
improved_predictions = best_random.predict(df_val)

In [ ]:
accuracy_score(labels_val_str, improved_predictions)

0.22901921132457026

In [ ]:
f1_score(labels_val_str, predictions, average= 'macro')

0.07118671577501716

In [ ]:
confusion_matrix(labels_val_str, predictions)

array([[  0,   0,   0,   1,   0,   3,   0,   0,   0,   0],
       [  0,   0,   0,  13,   0, 111,   0,   0,   0,   0],
       [ 52,   0,   0, 120,   5, 128,   0,   0,  30,   0],
       [  0,   1,   0, 341,   0, 151,   0,   0,   1,   0],
       [  4,   0,   0,  22,   0,  41,   0,   1,  14,   0],
       [  0,   0,   0,   9,   0,  95,   0,   0,   0,   0],
       [  1,   1,   0,  13,   0,  50,   0,   0,   0,   0],
       [ 25,   0,   0,  25,   0,  13,   0,   0,   8,   0],
       [ 29,   0,   0, 109,   0, 408,   0,   0,   0,   0],
       [  0,   1,   0,  98,   0,  54,   0,   0,   0,   0]])

SOmething is wrong with the predictions. None of the predictions match, and I am not able to see the matrix produced by predict and predic_proba

In [ ]:
bro = labels_str.to_numpy()

In [ ]:
X = input_df_val.to_numpy()

In [ ]:
hello_kitty = df_train.to_numpy()

In [ ]:
labels_trainable = labels_str.apply(int)

In [ ]:
labels_trainable = labels_trainable.to_numpy()

In [ ]:

classifier = MLkNN(k=5)

#train
classifier.fit(hello_kitty, labels_trainable )


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


IndexError: index (9917) out of range

check that is actually the training and not the validation

In [ ]:
labels_str

0        0
1        0
2        0
3        0
4        0
        ..
14110    8
14111    8
14112    8
14113    8
14114    8
Name: moa, Length: 14115, dtype: object

In [47]:

apply_class_weights = True
if apply_class_weights:     # if we want to apply class weights
    counts = labels_train_str.value_counts()  # count the number of moa in each class for the ENTiRE dataset
    print(counts)
    class_weights = []   # create list that will hold class weights
    for moa in labels_train_str.unique():       # for each moa   
        #print(moa)
        counts[moa]
        class_weights.append(counts[moa])  # add counts to class weights
    #print(len(class_weights))
    #print(class_weights)
    #print(type(class_weights))
    # class_weights = 1 / (class_weights / sum(class_weights)) # divide all class weights by total moas
    print(sum(class_weights))
    class_weights = [i / sum(class_weights) for  i in class_weights]
    class_weights_dict = {}
    for i, num in enumerate(class_weights):
        class_weights_dict[str(i)] = num
    class_weights_dict

3    3535
5    3363
0    1688
8    1592
1    1169
9     839
6     741
2     470
7     369
4     349
Name: moa, dtype: int64
14115


In [48]:
class_weights_dict

{'0': 0.1195890896209706,
 '1': 0.05944031172511512,
 '2': 0.02614240170031881,
 '3': 0.024725469358838115,
 '4': 0.03329791002479632,
 '5': 0.2504427913567127,
 '6': 0.23825717321997875,
 '7': 0.08281969535954659,
 '8': 0.05249734325185972,
 '9': 0.11278781438186326}

In [45]:
1688/14115

0.1195890896209706

In [46]:
class_weights

array([0.11958909, 0.05944031, 0.0261424 , 0.02472547, 0.03329791,
       0.25044279, 0.23825717, 0.0828197 , 0.05249734, 0.11278781])

In [ ]:
def applying_class_weights_funct(labels_train):
    ''' '''
    
    counts = labels_train.value_counts()  # count the number of moa in each class for the ENTiRE dataset
    class_weights = []   # create list that will hold class weights
    for moa in labels_train.unique():       # for each moa   
        print(moa)
        counts[moa]
        print(counts[moa])
        class_weights.append(counts[moa])  # add counts to class weights
    class_weights = [i / sum(class_weights) for  i in class_weights]
    class_weights_list = []
    for i, num in zip(labels_train.unique(), class_weights):
        class_weights_dict = {}
        print(num)
        cla
        class_weights_dict[i] = num
        class_weights_list.append(class_weights_dict)
    print(class_weights_list)
    return class_weights_list


In [52]:
 moas = input('List of moas (lst with strs) (ex: ["cyclooxygenase inhibitor", "adrenergic receptor antagonist"]): ') 


In [53]:
moas

'["cyclooxygenase inhibitor", "adrenergic receptor antagonist"]'